In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
# Creating path variable to read train data
RAW_DATA_PATH = os.path.abspath(os.path.join(os.getcwd(),os.path.pardir,'data','raw','train.csv'))

In [3]:
# Creating path variable to read test data
RAW_DATA_PATH_TEST = os.path.abspath(os.path.join(os.getcwd(),os.path.pardir,'data','raw','test.csv'))

In [4]:
# Creating package path
PACKAGE_PATH = os.path.abspath(os.path.join(os.getcwd(),os.path.pardir,'packages'))

In [5]:
# adding package path to path variable
sys.path.insert(1, PACKAGE_PATH)

In [6]:
# import functions from user defined packages
from eda import get_class_counts, normality_plots

In [7]:
from utils import load_all_transactions

In [9]:
df = load_all_transactions('train.csv')


Ther are a toral of 50660 transactions and Index(['ID', 'Agency', 'Agency Type', 'Distribution Channel', 'Product Name',
       'Claim', 'Duration', 'Destination', 'Net Sales', 'Commision (in value)',
       'Gender', 'Age'],
      dtype='object') features 



In [11]:
# having a peek at data
df.head()

,ID,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,3433,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,0,7,MALAYSIA,0.0,17.82,NaN,31
1,4339,EPX,Travel Agency,Online,Cancellation Plan,0,85,SINGAPORE,69.0,0.00,NaN,36
2,34590,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,0,11,MALAYSIA,19.8,11.88,NaN,75
3,55816,EPX,Travel Agency,Online,2 way Comprehensive Plan,0,16,INDONESIA,20.0,0.00,NaN,32
4,13816,EPX,Travel Agency,Online,Cancellation Plan,0,10,"KOREA, REPUBLIC OF",15.0,0.00,NaN,29


In [12]:
# getting info about training set
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50660 entries, 0 to 50659
Data columns (total 12 columns):
ID                      50660 non-null int64
Agency                  50660 non-null object
Agency Type             50660 non-null object
Distribution Channel    50660 non-null object
Product Name            50660 non-null object
Claim                   50660 non-null int64
Duration                50660 non-null int64
Destination             50660 non-null object
Net Sales               50660 non-null float64
Commision (in value)    50660 non-null float64
Gender                  14614 non-null object
Age                     50660 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 4.6+ MB


In [13]:
# checking is there any null values in the dataframe
df.isnull().sum()

ID                          0
Agency                      0
Agency Type                 0
Distribution Channel        0
Product Name                0
Claim                       0
Duration                    0
Destination                 0
Net Sales                   0
Commision (in value)        0
Gender                  36046
Age                         0
dtype: int64

**Insights**
1. There are null values in Gender column

In [14]:
df.nunique()

ID                      50660
Agency                     16
Agency Type                 2
Distribution Channel        2
Product Name               26
Claim                       2
Duration                  444
Destination               144
Net Sales                1053
Commision (in value)      968
Gender                      2
Age                        88
dtype: int64

**Insighst**
1. Agency type, distribution channel, gender are categorical variables. 
2. Claim is target variable

In [15]:
# Lest look at the Agency Type column
df['Agency Type'].value_counts()

Travel Agency    36672
Airlines         13988
Name: Agency Type, dtype: int64

**Insights**
1. 36,672 customers prefer to travel with Travel Agency
2. 13,988 (Approx. 14k) customers prefer to travel with Airlines

In [16]:
df['Distribution Channel'].value_counts()

Online     49770
Offline      890
Name: Distribution Channel, dtype: int64

**Insights**
1. Distribution of Travel Insurance is majorly via Online. 49770 customers have submitted claims Online.
2. Only 890 customers have submitted claims offline/personal visit.

In [17]:
# Lets count the % of claims rejected
print("Count of Rejection Claims - {}".format(round(df[df['Claim'] ==0].count()[0]/len(df)*100)),'%')

Count of Rejection Claims - 99.0 %


In [18]:
# Lets see the number of records in each class
pd.DataFrame(df.groupby(['Agency Type'])['Claim'].value_counts())

Claim
Agency Type   Claim       
Airlines      0      13514
              1        474
Travel Agency 0      36404
              1        268

**Insights**
1. Airlines has the highest number of Approved Claims 474 out of 13988 claim applications
2. Travel Agency has low number of Approved Claims 268 out of 36672 claim applications to Travel Agency

In [21]:
# Lets analyze the dataframe product wise
pd.DataFrame(df.groupby(['Product Name'])['Claim'].value_counts())

Claim
Product Name                         Claim       
1 way Comprehensive Plan             0       2647
                                     1          8
2 way Comprehensive Plan             0      10389
                                     1        112
24 Protect                           0        199
Annual Gold Plan                     0        131
                                     1         17
Annual Silver Plan                   0       1028
                                     1        128
Annual Travel Protect Gold           0         71
                                     1         10
Annual Travel Protect Platinum       0         44
                                     1          1
Annual Travel Protect Silver         0         69
                                     1          4
Basic Plan                           0       4364
                                     1         19
Bronze Plan                          0       3084
                                     1        162
Cancellation Plan                    0      14893
                                     1         33
Child Comprehensive Plan             0          7
Comprehensive Plan                   0        288
                                     1          5
Gold Plan                            0        278
                                     1         14
Individual Comprehensive Plan        0         55
                                     1          3
Premier Plan                         0        156
                                     1          3
Rental Vehicle Excess Insurance      0       6784
                                     1         68
Silver Plan                          0       1671
                                     1        118
Single Trip Travel Protect Gold      0        152
                                     1          7
Single Trip Travel Protect Platinum  0         54
                                     1          5
Single Trip Travel Protect Silver    0        132
                                     1          2
Spouse or Parents Comprehensive Plan 0         11
                                     1          1
Ticket Protector                     0        834
                                     1          5
Travel Cruise Protect                0        419
                                     1          2
Travel Cruise Protect Family         0          1
Value Plan                           0       2157
                                     1         15

**Insights**
1. Cancellation Plan has highest rejections of claims with 14893 rejected claims.

In [22]:
# finding % of nulls in gender column
nans = lambda df: df[df.isnull().any(axis=1)]
nan_percent = round((len(nans(df))/len(df))*100)
print(nan_percent,'% of data have NaN as Gender')

71 % of data have NaN as Gender


In [23]:
# Function to display all % of nulls in each group
def group_by_null(df, group_col, null_col):
    df2 = df[null_col].isnull().groupby([df[group_col]]).sum().astype(int).reset_index(name='count')
    df3 = pd.merge(df2,df[group_col].groupby([df[group_col]]).count().reset_index(name='total_count'),on=group_col)
    df3['null_%'] = (df3['count']/df3['total_count'])*100
    df4 = pd.DataFrame(df[df[null_col]=='F'].groupby([group_col])['ID'].count())
    df3 = pd.merge(df3,df4,on=group_col)
    df3.rename(columns={'ID':'Female_count'}, inplace=True)
    df4 = pd.DataFrame(df[df[null_col]=='M'].groupby([group_col])['ID'].count())
    df3 = pd.merge(df3,df4,on=group_col)
    df3.rename(columns={'ID':'Male_count'}, inplace=True)
    return df3

In [24]:
# finding no of nulls in each agency
group_by_null(df,'Agency','Gender')

,Agency,count,total_count,null_%,Female_count,Male_count
0,ADM,37,63,58.730159,13,13
1,ART,5,272,1.838235,81,186
2,C2B,0,6631,0.000000,3651,2980
3,CBH,0,81,0.000000,57,24
4,CCR,1,158,0.632911,116,41
5,CSR,0,68,0.000000,48,20
6,CWT,6847,6852,99.927029,3,2
7,JWT,15,606,2.475248,197,394
8,JZI,47,5069,0.927205,2051,2971
9,KML,0,318,0.000000,161,157
